<a href="https://colab.research.google.com/github/SeeUSoon93/MachineLerning/blob/main/ex03_%EB%8B%AE%EC%9D%80_%EA%BC%B4_%EC%95%B1_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# 현재 작업 디렉토리 확인
!pwd

/content


In [7]:
# 작업 디렉토리 변경
%cd "drive/MyDrive/Colab Notebooks/DeepLearning"

/content/drive/MyDrive/Colab Notebooks/DeepLearning


In [4]:
# 라이브러리, 모듈 로딩
import numpy as np
import pandas as pd
from tqdm import tqdm # 반복문의 진행률을 보여주는 도구
from tensorflow.keras.utils import image_dataset_from_directory # 폴더에서 이미지를 로딩해주는 모듈(함수)
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3 # 이미지 특징 추출 도구
from tensorflow.keras.models import Sequential # 모델의 뼈대를 만드는 도구
from tensorflow.keras.layers import Dense # 퍼셉트론(뉴런)의 묶음을 표현하는 클래스
from tensorflow.keras.callbacks import EarlyStopping # 조기학습중단
from tensorflow.keras.callbacks import ModelCheckpoint # 모델자동저장
from tensorflow.keras.models import load_model # hdf5 포멧의 모델파일을 로딩하는 함수
from sklearn.metrics import classification_report # 분류평가지표 확인 함수

In [ ]:
# 압축풀기
!unzip ./data/face.zip -d ./data/face/

In [9]:
total_data = image_dataset_from_directory(
    directory="./data/face/face",    # 읽을 폴더의 경로 설정
    labels = "inferred",    # 폴더명을 인식해서 정답으로 붙여준다.
    label_mode = "categorical",    # 다중분류 형태로 정답 생성
    color_mode = "rgb",    # 컬러사진 설정
    image_size = (224,224),    # 이미지 크기 리사이징
)

Found 3987 files belonging to 8 classes.


In [10]:
X_data = []
y_data = []

In [ ]:
for img, label in tqdm(total_data.as_numpy_iterator()) :
  X_data.append(img)
  y_data.append(label)

63it [01:42,  4.52s/it]

In [ ]:
# 리스트에 32장씩 담겨있는 사진데이터 및 정답데이터를 하나의 넘파이로 통합

X_numpy = np.concatenate(X_data)
y_numpy = np.concatenate(y_data)

In [ ]:
X_numpy.shape, y_numpy.shape

In [ ]:
# numpy 데이터 저장
# 매번 이미지를 읽어들이면 시간이 오래걸리기 때문에
# 읽어들인 numpy를 파일로 저장하여 시간을 단축
np.savez("./data/face/face.npz", X=X_numpy, y=y_numpy)

In [ ]:
npz_data = np.load("./data/face/face.npz")
X_numpy = npz_data['X']
y_numpy = npz_data['y']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_numpy, y_numpy, test_size=0.2, random_state=1026)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
# 불필요한 메모리 제거
del X_numpy
del y_numpy

In [ ]:
imageEmbedding = InceptionV3(
                 include_top = False,
                 weights="imagenet",
                 input_shape=(224,224,3), # 이미지 특징을 추출할 이미지 크기
                 pooling="avg")

In [ ]:
face_model = Sequential() # 뼈대 생성
face_model.add(imageEmbedding) # 이미지 특징 추출 도구 연결
face_model.add(Dense(units=128, activation="relu")) # 원하는만큼 퍼셉트론(뉴런) 연결
face_model.add(Dense(units=256, activation="relu")) # 뉴런이 많을수록 모델의 학습 능력 증가
face_model.add(Dense(units=128, activation="relu")) # 다만 모델이 무거워짐
face_model.add(Dense(units=8, activation="softmax")) # 정답 데이터의 종류(class 수)와 맞춤

In [ ]:
face_model.summary()

In [ ]:
face_model.compile(loss="sparse_categorical_crossentropy", # 모델의 예측과 실제정답의 차이를 계산하는 알고리즘
                     optimizer="adam", # 모델의 최적화를 도와주는 경사하강법 알고리즘의 한 종류
                     metrics=['accuracy'])

In [ ]:
# 조기학습 중단 : 모델이 일정 epoch내에 성능이 안 올라가면 자동으로 멈춰주는 기능(epoch 횟수 고민 X)
early = EarlyStopping(monitor="val_accuracy", # 모니터링 성능지표
                      patience=5) # 인내심 횟수

In [ ]:
# 모델 자동 저장 : 모델의 성능이 최고를 찍을 때 자동으로 중간중간 저장하는 기능(과대적합모델 생성방지)
model_path = "./data/face_model/face-{epoch:02d}-{val_accuracy:.2f}.hdf5" # 모델이 저장될 경로 및 파일 이름
mdckpt = ModelCheckpoint(filepath=model_path, # 파일경로 연결
                         save_best_only=True, # 최고점일 때만 자동저장
                         monitor="val_accuracy") # 최고점을 판단하는 성능지표

In [ ]:
face_model.fit(X_train, y_train, # 학습용 문제, 답 설정
                 validation_split = 0.2, # 검증용 데이터셋 비율 지정
                 epochs = 1000, # 학습 반복 횟수 설정
                 callbacks=[early, mdckpt], # 조기학습중단, 모델자동저장 연결
                 batch_size=16) # 한번에 RAM메모리에 적재되는 데이터 수

In [ ]:
# best 모델로딩
best_face_model = load_model("./data/face_model/face-21-0.78.hdf5")

In [ ]:
# 테스트 데이터 예측
pre = best_animal_model.predict(X_test)

In [ ]:
# 분류는
pre[0]

In [ ]:
# 분류평가지표 확인 (평가)
print(classification_report(y_test.argmax(axis=1), pre.argmax(axis=1))) # 실제정답, 모델의 예측값

In [ ]:
import gradio as gr
from typing_extensions import Doc

# 예측 알고리즘을 적용한 사용자 정의함수
def my_predict(input_data) :
  input_img = input_data.reshape(1,224,224,3)  # 1장의 이미지, 크기(224, 224), 컬러사진
  pre = best_face_model.predict(input_img)  # 매개변수로 들어온 이미지를 모델에게 예측시킴
  class_names = ["차은우","아이유","조인성","이정재","문재인","박해진","류준열","윤석열"]
  return class_names[pre.argmax(axis=1)[0]] # 확률중에 제일 높은 클래스번호를 글자로 변경

demo = gr.Interface(my_predict, # 예측 함수 연결
                    inputs= gr.Image(shape=(224,224)), # 생성될 입력 인터페이스 설정
                    outputs= "text") # 생성될 출력 인터페이스 설정

# 앱 실행
demo.launch(share=True) # share 옵션 : 외부에서 접속 가능한 URL생성 (일시적)